# HIFI PART ANALYSIS

In [1]:
import  pandas
import re
from pygal.style import Style
import pygal
from IPython.display import SVG, display

In [2]:
original = pandas.read_csv(
"https://docs.google.com/spreadsheets/d/1ZlTOYxmPs938gqHjtDABkWS-MApu7uJjzIGnJ2Elm6Y/export?gid=1817590117&format=csv")

In [3]:
data = original

In [4]:
data["Power (W)\n(1% THD+N, 4 ohms)"] = data["Power (W)\n(1% THD+N, 4 ohms)"].apply(lambda x: float(x.replace(",", ""))
                                                                                   )

In [5]:
list(data.columns)

['Name',
 'SINAD (dB)\n(5W, 4 ohms)',
 'Price (USD)',
 'Less than 150W',
 'Between 150 and 300 W',
 'Between 300 and 450W',
 'Between 450 and 600W',
 'More than 600W',
 'Power (W)\n(1% THD+N, 4 ohms)']

In [6]:
#  rename columns

data = data[
    [
        "Name", "SINAD (dB)\n(5W, 4 ohms)", "Price (USD)", "Power (W)\n(1% THD+N, 4 ohms)"
    ]
]


In [7]:
data.columns = [
    "name",  "sinad_5w_4ohm", "price_usd", "thd_1pc_4ohm"
]

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 634 entries, 0 to 633
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           634 non-null    object 
 1   sinad_5w_4ohm  634 non-null    int64  
 2   price_usd      632 non-null    object 
 3   thd_1pc_4ohm   634 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 19.9+ KB


In [9]:
describe = data.describe()
describe

,sinad_5w_4ohm,thd_1pc_4ohm
count,634.000000,634.000000
mean,77.175079,325.113565
std,16.287095,326.834847
min,23.000000,0.000000
25%,66.000000,115.250000
50%,76.000000,230.000000
75%,88.000000,420.000000
max,121.000000,2500.000000


In [10]:
###  can use these quartiles to divide thd column  into 4 categories

first_quartile_thd_bound = describe.loc["25%", "thd_1pc_4ohm"]
second_quartile_thd_bound = describe.loc["50%", "thd_1pc_4ohm"]
third_quartile_thd_bound = describe.loc["75%", "thd_1pc_4ohm"]

In [11]:
# an outlier is any number less than or greater than . 
interquartile_range = third_quartile_thd_bound - first_quartile_thd_bound
lower_bound_outlier = first_quartile_thd_bound - (1.5 * interquartile_range)
upper_bound_outlier = third_quartile_thd_bound + (1.5 * interquartile_range)

def is_thd_outlier(thd):
    if thd > upper_bound_outlier or thd < lower_bound_outlier:
        return True
    return False

In [12]:
quartile_dict = {
    "Q1": f"0 - {first_quartile_thd_bound} watts",
    "Q2": f"{first_quartile_thd_bound} watts - {second_quartile_thd_bound} watts",
    "Q3": f"{second_quartile_thd_bound} watts - {third_quartile_thd_bound} watts",
    "Q4": f"{third_quartile_thd_bound} watts and above"
}

def parse_thd_category(thd):
    if (thd < first_quartile_thd_bound):
        return quartile_dict["Q1"]
    elif (thd < second_quartile_thd_bound):
        return quartile_dict["Q2"]
    elif (thd < third_quartile_thd_bound):
        return quartile_dict["Q3"]
    return quartile_dict["Q4"]

def parse_thd_category_as_quartile(thd):
    if (thd < first_quartile_thd_bound):
        return "quartile 1"
    elif (thd < second_quartile_thd_bound):
        return "quartile 2"
    elif (thd < third_quartile_thd_bound):
        return "quartile 3"
    return "quartile 4"

In [13]:
def parse_usd(price):
  
    if type(price) == float:
        return price
    price_no_commas = price.replace(",", "")
    parsed = re.match("^\d+", price_no_commas)
    if (parsed is None):
        return -1
    return float(price_no_commas)

In [14]:
def display_and_export_ranked_horizontal_bar_chart(chart_data, chart_title, x_axis, y_axis, export_location):
    values = list(zip(chart_data[y_axis], chart_data[x_axis]))
    style = Style(
        colors=tuple(["#4AC5D8" for _ in range(chart_data.shape[0])])
    )
    
    chart = pygal.HorizontalBar(style=style,
                                legend_at_bottom=True,
                                show_legend=False,
                                human_readable=True, 
                                print_values=True,
                                print_labels=True
    )
    chart.title = chart_title

    # order needs to be reversed because pygal inserts data from bottom to top
    chart.x_labels = list(reversed(list(chart_data[y_axis])))
    chart.add("", list(reversed(list(chart_data[x_axis]))))

    chart.render_to_file(export_location)
    # display(SVG(chart.render(disable_xml_declaration=True)))

In [15]:
# convert  the price column to float values and get rid of entries without a price
data["price_usd"] = data["price_usd"].apply(parse_usd)
data = data[data["price_usd"] > 0]

C:\Users\danie\AppData\Local\Temp\ipykernel_14212\236475459.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["price_usd"] = data["price_usd"].apply(parse_usd)


In [16]:
data["thd_category"] = data["thd_1pc_4ohm"].apply(parse_thd_category)
data["thd_quartile"] = data["thd_1pc_4ohm"].apply(parse_thd_category_as_quartile)
data["is_thd_outlier"] = data["thd_1pc_4ohm"].apply(is_thd_outlier)

In [17]:
data["price_to_sinad"] = data.apply(lambda x: round(x.sinad_5w_4ohm / x.price_usd, 2), axis=1)
data["price_to_thd_1pc"] = data.apply(lambda x: round(x.thd_1pc_4ohm / x.price_usd, 2), axis=1)

## THD @ 1% 4ohm analysis

### Top 10 price to thd 1% regardless of price

In [18]:
subset = data.sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
186,Pascal X-PRO1,86,700.0,2000.0,420.0 watts and above,quartile 4,True,0.12,2.86
581,Rockville RPA16,54,390.0,885.0,420.0 watts and above,quartile 4,True,0.14,2.27
149,Fosi Audio V3 (48V PS),88,110.0,160.0,115.25 watts - 230.0 watts,quartile 2,False,0.80,1.45
323,Fosi Audio TB10D,76,70.0,98.0,0 - 115.25 watts,quartile 1,False,1.09,1.40
225,Fosi Audio BT20A Pro (48 V),83,140.0,180.0,115.25 watts - 230.0 watts,quartile 2,False,0.59,1.29
442,Behringer NX3000D,69,390.0,500.0,420.0 watts and above,quartile 4,False,0.18,1.28
118,ICEpower 700AS2,92,576.0,720.0,420.0 watts and above,quartile 4,False,0.16,1.25
418,Crown XLS 2502,71,650.0,800.0,420.0 watts and above,quartile 4,False,0.11,1.23
222,AIYIMA A07 Pro,83,90.0,108.0,0 - 115.25 watts,quartile 1,False,0.92,1.20
235,AIYIMA A07 TPA3255,83,66.0,77.0,0 - 115.25 watts,quartile 1,False,1.26,1.17


In [19]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% regardless of price',
    export_location='./chart-exports/top-10-price_to_thd_1pc-unfiltered.svg'
)

<img src="chart-exports/top-10-price_to_thd_1pc-unfiltered.svg" width="1000"/>

### Top 10 price to thd 1%,  under 125USD

In [20]:
subset = data[data["price_usd"] < 125].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
149,Fosi Audio V3 (48V PS),88,110.0,160.0,115.25 watts - 230.0 watts,quartile 2,False,0.80,1.45
323,Fosi Audio TB10D,76,70.0,98.0,0 - 115.25 watts,quartile 1,False,1.09,1.40
222,AIYIMA A07 Pro,83,90.0,108.0,0 - 115.25 watts,quartile 1,False,0.92,1.20
235,AIYIMA A07 TPA3255,83,66.0,77.0,0 - 115.25 watts,quartile 1,False,1.26,1.17
497,UWAYKEY ZK-502H,64,18.0,20.0,0 - 115.25 watts,quartile 1,False,3.56,1.11
234,AIYIMA A08 Pro,83,106.0,115.0,0 - 115.25 watts,quartile 1,False,0.78,1.08
375,AIYIMA A05,74,80.0,80.0,0 - 115.25 watts,quartile 1,False,0.93,1.00
148,Fosi Audio V3 (32V PS),88,90.0,85.0,0 - 115.25 watts,quartile 1,False,0.98,0.94
324,Sabaj A1 (2022),76,70.0,55.0,0 - 115.25 watts,quartile 1,False,1.09,0.79
224,Fosi Audio BT20A Pro (32 V),83,100.0,77.0,0 - 115.25 watts,quartile 1,False,0.83,0.77


In [21]:

display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% under $125 USD',
    export_location='./chart-exports/top-10-price_to_thd_1pc-under-125-usd.svg'
)

<img src="chart-exports/top-10-price_to_thd_1pc-under-125-usd.svg" width="1000"/>

### Top 10 price to thd 1%,  under 250USD

In [22]:
subset = data[data["price_usd"] < 250].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
149,Fosi Audio V3 (48V PS),88,110.0,160.0,115.25 watts - 230.0 watts,quartile 2,False,0.80,1.45
323,Fosi Audio TB10D,76,70.0,98.0,0 - 115.25 watts,quartile 1,False,1.09,1.40
225,Fosi Audio BT20A Pro (48 V),83,140.0,180.0,115.25 watts - 230.0 watts,quartile 2,False,0.59,1.29
222,AIYIMA A07 Pro,83,90.0,108.0,0 - 115.25 watts,quartile 1,False,0.92,1.20
235,AIYIMA A07 TPA3255,83,66.0,77.0,0 - 115.25 watts,quartile 1,False,1.26,1.17
497,UWAYKEY ZK-502H,64,18.0,20.0,0 - 115.25 watts,quartile 1,False,3.56,1.11
234,AIYIMA A08 Pro,83,106.0,115.0,0 - 115.25 watts,quartile 1,False,0.78,1.08
376,Monoprice Studio 605030,74,152.0,157.0,115.25 watts - 230.0 watts,quartile 2,False,0.49,1.03
460,Behringer A500,67,200.0,200.0,115.25 watts - 230.0 watts,quartile 2,False,0.34,1.00
375,AIYIMA A05,74,80.0,80.0,0 - 115.25 watts,quartile 1,False,0.93,1.00


In [23]:

display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% under $250 USD',
    export_location='./chart-exports/top-10-price_to_thd_1pc-under-250-usd.svg'
)

<img src="chart-exports/top-10-price_to_thd_1pc-under-250-usd.svg" width="1000"/>

### Top 10 price to thd 1%,  under 500USD

In [24]:
subset = data[data["price_usd"] < 500].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
581,Rockville RPA16,54,390.0,885.0,420.0 watts and above,quartile 4,True,0.14,2.27
149,Fosi Audio V3 (48V PS),88,110.0,160.0,115.25 watts - 230.0 watts,quartile 2,False,0.80,1.45
323,Fosi Audio TB10D,76,70.0,98.0,0 - 115.25 watts,quartile 1,False,1.09,1.40
225,Fosi Audio BT20A Pro (48 V),83,140.0,180.0,115.25 watts - 230.0 watts,quartile 2,False,0.59,1.29
442,Behringer NX3000D,69,390.0,500.0,420.0 watts and above,quartile 4,False,0.18,1.28
222,AIYIMA A07 Pro,83,90.0,108.0,0 - 115.25 watts,quartile 1,False,0.92,1.20
235,AIYIMA A07 TPA3255,83,66.0,77.0,0 - 115.25 watts,quartile 1,False,1.26,1.17
567,Crown XLi 800,56,270.0,315.0,230.0 watts - 420.0 watts,quartile 3,False,0.21,1.17
497,UWAYKEY ZK-502H,64,18.0,20.0,0 - 115.25 watts,quartile 1,False,3.56,1.11
234,AIYIMA A08 Pro,83,106.0,115.0,0 - 115.25 watts,quartile 1,False,0.78,1.08


In [25]:

display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% under $500 USD',
    export_location='./chart-exports/top-10-price_to_thd_1pc-under-500-usd.svg'
)

<img src="chart-exports/top-10-price_to_thd_1pc-under-500-usd.svg" width="1000"/>

### Top 10 price to thd 1%,  under 1000USD

In [26]:
subset = data[data["price_usd"] < 1000].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
186,Pascal X-PRO1,86,700.0,2000.0,420.0 watts and above,quartile 4,True,0.12,2.86
581,Rockville RPA16,54,390.0,885.0,420.0 watts and above,quartile 4,True,0.14,2.27
149,Fosi Audio V3 (48V PS),88,110.0,160.0,115.25 watts - 230.0 watts,quartile 2,False,0.80,1.45
323,Fosi Audio TB10D,76,70.0,98.0,0 - 115.25 watts,quartile 1,False,1.09,1.40
225,Fosi Audio BT20A Pro (48 V),83,140.0,180.0,115.25 watts - 230.0 watts,quartile 2,False,0.59,1.29
442,Behringer NX3000D,69,390.0,500.0,420.0 watts and above,quartile 4,False,0.18,1.28
118,ICEpower 700AS2,92,576.0,720.0,420.0 watts and above,quartile 4,False,0.16,1.25
418,Crown XLS 2502,71,650.0,800.0,420.0 watts and above,quartile 4,False,0.11,1.23
222,AIYIMA A07 Pro,83,90.0,108.0,0 - 115.25 watts,quartile 1,False,0.92,1.20
235,AIYIMA A07 TPA3255,83,66.0,77.0,0 - 115.25 watts,quartile 1,False,1.26,1.17


In [27]:

display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% under $1000 USD',
    export_location='./chart-exports/top-10-price_to_thd_1pc-under-1000-usd.svg'
)

<img src="chart-exports/top-10-price_to_thd_1pc-under-1000-usd.svg" width="1000"/>

### Top 10 price to thd 1%, in first quartile (thd_1pc_4ohm) category

In [28]:
subset = data[
    data["thd_quartile"] == "quartile 1"
].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset 

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
323,Fosi Audio TB10D,76,70.0,98.0,0 - 115.25 watts,quartile 1,False,1.09,1.40
222,AIYIMA A07 Pro,83,90.0,108.0,0 - 115.25 watts,quartile 1,False,0.92,1.20
235,AIYIMA A07 TPA3255,83,66.0,77.0,0 - 115.25 watts,quartile 1,False,1.26,1.17
497,UWAYKEY ZK-502H,64,18.0,20.0,0 - 115.25 watts,quartile 1,False,3.56,1.11
234,AIYIMA A08 Pro,83,106.0,115.0,0 - 115.25 watts,quartile 1,False,0.78,1.08
375,AIYIMA A05,74,80.0,80.0,0 - 115.25 watts,quartile 1,False,0.93,1.00
148,Fosi Audio V3 (32V PS),88,90.0,85.0,0 - 115.25 watts,quartile 1,False,0.98,0.94
324,Sabaj A1 (2022),76,70.0,55.0,0 - 115.25 watts,quartile 1,False,1.09,0.79
224,Fosi Audio BT20A Pro (32 V),83,100.0,77.0,0 - 115.25 watts,quartile 1,False,0.83,0.77
290,Douk Audio H7,78,144.0,103.0,0 - 115.25 watts,quartile 1,False,0.54,0.72


In [29]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% first quartile',
    export_location='./chart-exports/top-10-price-to-thd-1pc-first-quartile.svg'
)

<img src="chart-exports/top-10-price-to-thd-1pc-first-quartile.svg" width="1000"/>

### Top 10 price to thd 1%, in second quartile (thd_1pc_4ohm) category

In [30]:
subset = data[
    data["thd_quartile"] == "quartile 2"
].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
149,Fosi Audio V3 (48V PS),88,110.0,160.0,115.25 watts - 230.0 watts,quartile 2,False,0.80,1.45
225,Fosi Audio BT20A Pro (48 V),83,140.0,180.0,115.25 watts - 230.0 watts,quartile 2,False,0.59,1.29
376,Monoprice Studio 605030,74,152.0,157.0,115.25 watts - 230.0 watts,quartile 2,False,0.49,1.03
460,Behringer A500,67,200.0,200.0,115.25 watts - 230.0 watts,quartile 2,False,0.34,1.00
160,Fosi Audio ZA3 48V PS,88,150.0,147.0,115.25 watts - 230.0 watts,quartile 2,False,0.59,0.98
326,Hypex UcD180HG HxR,76,135.0,130.0,115.25 watts - 230.0 watts,quartile 2,False,0.56,0.96
452,Pyle PT8000CH 8-Channel,68,270.0,227.0,115.25 watts - 230.0 watts,quartile 2,False,0.25,0.84
300,Yamaha R-S202,78,200.0,163.0,115.25 watts - 230.0 watts,quartile 2,False,0.39,0.81
81,AIYIMA A70 PS 48V 10A,95,220.0,178.0,115.25 watts - 230.0 watts,quartile 2,False,0.43,0.81
256,Niles SI-275,81,230.0,147.0,115.25 watts - 230.0 watts,quartile 2,False,0.35,0.64


In [31]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% second quartile',
    export_location='./chart-exports/top-10-price-to-thd-1pc-second-quartile.svg'
)

<img src="chart-exports/top-10-price-to-thd-1pc-second-quartile.svg" width="1000"/>

### Top 10 price to thd 1%, in third quartile (thd_1pc_4ohm) category

In [32]:
subset = data[
    data["thd_quartile"] == "quartile 3"
].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
567,Crown XLi 800,56,270.0,315.0,230.0 watts - 420.0 watts,quartile 3,False,0.21,1.17
310,Behringer A800,77,300.0,310.0,230.0 watts - 420.0 watts,quartile 3,False,0.26,1.03
350,Crown XLS 1002,75,340.0,330.0,230.0 watts - 420.0 watts,quartile 3,False,0.22,0.97
56,Fosi Audio V3 Mono,100,280.0,242.0,230.0 watts - 420.0 watts,quartile 3,False,0.36,0.86
123,ICEpower 300AS1,91,435.0,310.0,230.0 watts - 420.0 watts,quartile 3,False,0.21,0.71
114,ICEpower 125ASX2 (bridge),92,540.0,370.0,230.0 watts - 420.0 watts,quartile 3,False,0.17,0.69
109,NAD 2200 normal in,93,530.0,323.0,230.0 watts - 420.0 watts,quartile 3,False,0.18,0.61
82,NAD 2200 lab in,95,530.0,323.0,230.0 watts - 420.0 watts,quartile 3,False,0.18,0.61
63,Hypex NCore NC252MP,98,430.0,250.0,230.0 watts - 420.0 watts,quartile 3,False,0.23,0.58
419,Dayton Audio APA1200DSP,71,650.0,354.0,230.0 watts - 420.0 watts,quartile 3,False,0.11,0.54


In [33]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% third quartile',
    export_location='./chart-exports/top-10-price-to-thd-1pc-third-quartile.svg'
)

<img src="chart-exports/top-10-price-to-thd-1pc-third-quartile.svg" width="1000"/>

### Top 10 price to thd 1%, in fourth quartile (thd_1pc_4ohm) category

In [34]:
subset = data[
    data["thd_quartile"] == "quartile 4"
].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset 

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
186,Pascal X-PRO1,86,700.0,2000.0,420.0 watts and above,quartile 4,True,0.12,2.86
581,Rockville RPA16,54,390.0,885.0,420.0 watts and above,quartile 4,True,0.14,2.27
442,Behringer NX3000D,69,390.0,500.0,420.0 watts and above,quartile 4,False,0.18,1.28
118,ICEpower 700AS2,92,576.0,720.0,420.0 watts and above,quartile 4,False,0.16,1.25
418,Crown XLS 2502,71,650.0,800.0,420.0 watts and above,quartile 4,False,0.11,1.23
66,ICEpower 2000AS2 HV,98,898.0,1000.0,420.0 watts and above,quartile 4,True,0.11,1.11
19,Hypex NCore NC2K OEM,107,2360.0,2500.0,420.0 watts and above,quartile 4,True,0.05,1.06
337,ICEpower 1000ASP,75,1136.0,1175.0,420.0 watts and above,quartile 4,True,0.07,1.03
329,Crown XLS 1502,76,500.0,500.0,420.0 watts and above,quartile 4,False,0.15,1.00
90,ICEpower 1200AS2,94,745.0,670.0,420.0 watts and above,quartile 4,False,0.13,0.90


In [35]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% fourth quartile',
    export_location='./chart-exports/top-10-price-to-thd-1pc-fourth-quartile.svg'
)

<img src="chart-exports/top-10-price-to-thd-1pc-fourth-quartile.svg" width="1000"/>

### Top 10 price to thd 1%, where sinad is >= 80

In [36]:
subset = data[
    data["sinad_5w_4ohm"] >= 80
].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
186,Pascal X-PRO1,86,700.0,2000.0,420.0 watts and above,quartile 4,True,0.12,2.86
149,Fosi Audio V3 (48V PS),88,110.0,160.0,115.25 watts - 230.0 watts,quartile 2,False,0.80,1.45
225,Fosi Audio BT20A Pro (48 V),83,140.0,180.0,115.25 watts - 230.0 watts,quartile 2,False,0.59,1.29
118,ICEpower 700AS2,92,576.0,720.0,420.0 watts and above,quartile 4,False,0.16,1.25
222,AIYIMA A07 Pro,83,90.0,108.0,0 - 115.25 watts,quartile 1,False,0.92,1.20
235,AIYIMA A07 TPA3255,83,66.0,77.0,0 - 115.25 watts,quartile 1,False,1.26,1.17
66,ICEpower 2000AS2 HV,98,898.0,1000.0,420.0 watts and above,quartile 4,True,0.11,1.11
234,AIYIMA A08 Pro,83,106.0,115.0,0 - 115.25 watts,quartile 1,False,0.78,1.08
19,Hypex NCore NC2K OEM,107,2360.0,2500.0,420.0 watts and above,quartile 4,True,0.05,1.06
160,Fosi Audio ZA3 48V PS,88,150.0,147.0,115.25 watts - 230.0 watts,quartile 2,False,0.59,0.98


In [37]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1%, filtered by sinad >= 80',
    export_location='./chart-exports/top-10-price-to-thd-1pc-sinad-over-equal-80.svg'
)

<img src="chart-exports/top-10-price-to-thd-1pc-sinad-over-equal-80.svg" width="1000"/>

### Top 10 price to thd 1%, where sinad is >= 100

In [38]:
subset = data[
    data["sinad_5w_4ohm"] >= 100
].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
19,Hypex NCore NC2K OEM,107,2360.0,2500.0,420.0 watts and above,quartile 4,True,0.05,1.06
56,Fosi Audio V3 Mono,100,280.0,242.0,230.0 watts - 420.0 watts,quartile 3,False,0.36,0.86
50,Hypex NCore NC502MP,101,665.0,500.0,420.0 watts and above,quartile 4,False,0.15,0.75
22,Hypex NCoreX NCx500 OEM (buffered),106,970.0,656.0,420.0 watts and above,quartile 4,False,0.11,0.68
8,Hypex NCoreX NCx500 OEM (bufferless),113,990.0,656.0,420.0 watts and above,quartile 4,False,0.11,0.66
12,Apollon NCx500ST (low gain),112,1165.0,644.0,420.0 watts and above,quartile 4,False,0.10,0.55
43,3e audio TPA3255 260-2-29A,102,375.0,206.0,115.25 watts - 230.0 watts,quartile 2,False,0.27,0.55
28,Topping PA5 II,105,220.0,120.0,115.25 watts - 230.0 watts,quartile 2,False,0.48,0.55
38,Hypex NCore NC1200 OEM,103,1390.0,700.0,420.0 watts and above,quartile 4,False,0.07,0.50
29,TP RA3,105,269.0,131.0,115.25 watts - 230.0 watts,quartile 2,False,0.39,0.49


In [39]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1%, filtered by sinad >= 100',
    export_location='./chart-exports/top-10-price-to-thd-1pc-sinad-over-equal-100.svg'
)

<img src="chart-exports/top-10-price-to-thd-1pc-sinad-over-equal-100.svg" width="1000"/>

## Sinad analysis

### Top 20 sinad performance overall

In [40]:
subset = data.sort_values("sinad_5w_4ohm", ascending=False).head(20)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
0,"Topping LA90 (stereo, bypass, low gain)",121,800.0,90.0,0 - 115.25 watts,quartile 1,False,0.15,0.11
1,"Topping LA90 Discrete (bridged, low gain)",120,1600.0,320.0,230.0 watts - 420.0 watts,quartile 3,False,0.07,0.20
2,"Topping LA90 Discrete (stereo, low gain)",120,800.0,91.0,0 - 115.25 watts,quartile 1,False,0.15,0.11
3,"Topping LA90 (bridged, bypass, low gain)",119,1600.0,180.0,115.25 watts - 230.0 watts,quartile 2,False,0.07,0.11
4,"Topping LA90 Discrete (stereo, high gain)",117,800.0,91.0,0 - 115.25 watts,quartile 1,False,0.15,0.11
5,"Topping LA90 (stereo, bypass, high gain)",117,800.0,90.0,0 - 115.25 watts,quartile 1,False,0.15,0.11
6,Purifi 1ET9040BA,117,2500.0,900.0,420.0 watts and above,quartile 4,True,0.05,0.36
7,"Topping LA90 (stereo, volume, low gain)",115,800.0,90.0,0 - 115.25 watts,quartile 1,False,0.14,0.11
8,Hypex NCoreX NCx500 OEM (bufferless),113,990.0,656.0,420.0 watts and above,quartile 4,False,0.11,0.66
9,Hypex Nilai500DIY stereo kit (low gain),113,1325.0,300.0,230.0 watts - 420.0 watts,quartile 3,False,0.09,0.23


In [41]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='sinad_5w_4ohm',
    y_axis='name',
    chart_title='top 20 sinad_5w_4ohm performance overall',
    export_location='./chart-exports/top-20-sinad_5w_4ohm-performance-overall.svg'
)

<img src="chart-exports/top-20-sinad_5w_4ohm-performance-overall.svg" width="1000"/>

### Top 20 sinad per dollar performance, overall

In [42]:
subset = data.sort_values("price_to_sinad", ascending=False).head(20)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
497,UWAYKEY ZK-502H,64,18.0,20.0,0 - 115.25 watts,quartile 1,False,3.56,1.11
235,AIYIMA A07 TPA3255,83,66.0,77.0,0 - 115.25 watts,quartile 1,False,1.26,1.17
449,Sabaj A1 (2020),68,60.0,22.0,0 - 115.25 watts,quartile 1,False,1.13,0.37
323,Fosi Audio TB10D,76,70.0,98.0,0 - 115.25 watts,quartile 1,False,1.09,1.40
324,Sabaj A1 (2022),76,70.0,55.0,0 - 115.25 watts,quartile 1,False,1.09,0.79
484,Fosi Audio BT10A,65,60.0,35.0,0 - 115.25 watts,quartile 1,False,1.08,0.58
148,Fosi Audio V3 (32V PS),88,90.0,85.0,0 - 115.25 watts,quartile 1,False,0.98,0.94
375,AIYIMA A05,74,80.0,80.0,0 - 115.25 watts,quartile 1,False,0.93,1.00
222,AIYIMA A07 Pro,83,90.0,108.0,0 - 115.25 watts,quartile 1,False,0.92,1.20
224,Fosi Audio BT20A Pro (32 V),83,100.0,77.0,0 - 115.25 watts,quartile 1,False,0.83,0.77


In [43]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_sinad',
    y_axis='name',
    chart_title='top 20 price_to_sinad',
    export_location='./chart-exports/top-20-price_to_sinad.svg'
)

<img src="chart-exports/top-20-price_to_sinad.svg" width="1000"/>

### Top 10 sinad per dollar performance, where sinad is > 80

In [44]:
subset = data[data["sinad_5w_4ohm"]  >= 80].sort_values("price_to_sinad", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
235,AIYIMA A07 TPA3255,83,66.0,77.0,0 - 115.25 watts,quartile 1,False,1.26,1.17
148,Fosi Audio V3 (32V PS),88,90.0,85.0,0 - 115.25 watts,quartile 1,False,0.98,0.94
222,AIYIMA A07 Pro,83,90.0,108.0,0 - 115.25 watts,quartile 1,False,0.92,1.20
224,Fosi Audio BT20A Pro (32 V),83,100.0,77.0,0 - 115.25 watts,quartile 1,False,0.83,0.77
254,SMSL DA-6,81,100.0,39.0,0 - 115.25 watts,quartile 1,False,0.81,0.39
149,Fosi Audio V3 (48V PS),88,110.0,160.0,115.25 watts - 230.0 watts,quartile 2,False,0.80,1.45
234,AIYIMA A08 Pro,83,106.0,115.0,0 - 115.25 watts,quartile 1,False,0.78,1.08
275,3e Audio SY-DAP2002 (LM4562 upgrade),80,116.0,75.0,0 - 115.25 watts,quartile 1,False,0.69,0.65
218,Allo Volt+ D,84,129.0,62.0,0 - 115.25 watts,quartile 1,False,0.65,0.48
122,ICEpower 50ASX2 (stereo),91,147.0,50.0,0 - 115.25 watts,quartile 1,False,0.62,0.34


In [45]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_sinad',
    y_axis='name',
    chart_title='top 10 sinad per dollar performance >80 sinad',
    export_location='./chart-exports/top-10-sinad-per-dollar-performance-where-sinad-greater-than-80.svg'
)

<img src="chart-exports/top-10-sinad-per-dollar-performance-where-sinad-greater-than-80.svg" width="1000"/>

### Top 10 sinad per dollar performance, where sinad is > 100

In [46]:
subset = data[data["sinad_5w_4ohm"]  >= 100].sort_values("price_to_sinad", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
28,Topping PA5 II,105,220.0,120.0,115.25 watts - 230.0 watts,quartile 2,False,0.48,0.55
29,TP RA3,105,269.0,131.0,115.25 watts - 230.0 watts,quartile 2,False,0.39,0.49
56,Fosi Audio V3 Mono,100,280.0,242.0,230.0 watts - 420.0 watts,quartile 3,False,0.36,0.86
23,Topping PA5,106,350.0,131.0,115.25 watts - 230.0 watts,quartile 2,False,0.30,0.37
49,Hypex NCore NC52MP,101,345.0,50.0,0 - 115.25 watts,quartile 1,False,0.29,0.14
43,3e audio TPA3255 260-2-29A,102,375.0,206.0,115.25 watts - 230.0 watts,quartile 2,False,0.27,0.55
44,Topping PA7 Plus,102,550.0,240.0,230.0 watts - 420.0 watts,quartile 3,False,0.19,0.44
57,March Audio P122,100,595.0,126.0,115.25 watts - 230.0 watts,quartile 2,False,0.17,0.21
50,Hypex NCore NC502MP,101,665.0,500.0,420.0 watts and above,quartile 4,False,0.15,0.75
0,"Topping LA90 (stereo, bypass, low gain)",121,800.0,90.0,0 - 115.25 watts,quartile 1,False,0.15,0.11


In [47]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_sinad',
    y_axis='name',
    chart_title='top 10 sinad per dollar performance >100 sinad',
    export_location='./chart-exports/top-10-sinad-per-dollar-performance-where-sinad-greater-than-100.svg'
)

<img src="chart-exports/top-10-sinad-per-dollar-performance-where-sinad-greater-than-100.svg" width="1000"/>

## Price analysis

### Most expensive 20

In [48]:
subset = data.sort_values("price_usd", ascending=False).head(20)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
553,darTZeel NHB-468,59,170000.0,205.0,115.25 watts - 230.0 watts,quartile 2,False,0.0,0.00
613,darTZeel NHB-458,48,144500.0,900.0,420.0 watts and above,quartile 4,True,0.0,0.01
284,Simaudio Moon 888,80,118888.0,1342.0,420.0 watts and above,quartile 4,True,0.0,0.01
532,Karan Acoustics Master Collection POWERa Mono,61,106000.0,2500.0,420.0 watts and above,quartile 4,True,0.0,0.02
71,MBL Reference 9011 monoblock,97,106000.0,870.0,420.0 watts and above,quartile 4,False,0.0,0.01
184,Gryphon Apex Stereo,86,99000.0,450.0,420.0 watts and above,quartile 4,False,0.0,0.00
183,Boulder Amplifiers 2150,86,99000.0,1450.0,420.0 watts and above,quartile 4,True,0.0,0.01
55,Linn Klimax Solo 800,100,90000.0,805.0,420.0 watts and above,quartile 4,False,0.0,0.01
565,Dan D'Agostino Momentum M400 MxV monoblock,58,80000.0,778.0,420.0 watts and above,quartile 4,False,0.0,0.01
580,Octave Audio Jubilee Mono SE,55,80000.0,68.0,0 - 115.25 watts,quartile 1,False,0.0,0.00


In [49]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_usd',
    y_axis='name',
    chart_title='top 20 by price_usd',
    export_location='./chart-exports/top-20-expensive-by-price-usd.svg'
)

<img src="chart-exports/top-20-expensive-by-price-usd.svg" width="1000"/>

### Cheapest 20

In [50]:
subset = data.sort_values("price_usd", ascending=True).head(20)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
497,UWAYKEY ZK-502H,64,18.0,20.0,0 - 115.25 watts,quartile 1,False,3.56,1.11
449,Sabaj A1 (2020),68,60.0,22.0,0 - 115.25 watts,quartile 1,False,1.13,0.37
484,Fosi Audio BT10A,65,60.0,35.0,0 - 115.25 watts,quartile 1,False,1.08,0.58
235,AIYIMA A07 TPA3255,83,66.0,77.0,0 - 115.25 watts,quartile 1,False,1.26,1.17
324,Sabaj A1 (2022),76,70.0,55.0,0 - 115.25 watts,quartile 1,False,1.09,0.79
323,Fosi Audio TB10D,76,70.0,98.0,0 - 115.25 watts,quartile 1,False,1.09,1.40
583,SMSL SA100,54,74.0,22.0,0 - 115.25 watts,quartile 1,False,0.73,0.30
375,AIYIMA A05,74,80.0,80.0,0 - 115.25 watts,quartile 1,False,0.93,1.00
148,Fosi Audio V3 (32V PS),88,90.0,85.0,0 - 115.25 watts,quartile 1,False,0.98,0.94
222,AIYIMA A07 Pro,83,90.0,108.0,0 - 115.25 watts,quartile 1,False,0.92,1.20


In [51]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_usd',
    y_axis='name',
    chart_title='top 20 cheapest (price_usd)',
    export_location='./chart-exports/top-20-cheapest-by-price-usd.svg'
)

<img src="chart-exports/top-20-cheapest-by-price-usd.svg" width="1000"/>

## Additional Charts

### Sinad histogram

In [52]:
data["sinad_5w_4ohm_integer"] = data["sinad_5w_4ohm"].apply(lambda x: int(x))

In [53]:
histo_data = data.groupby(by="sinad_5w_4ohm_integer").size().reset_index()

In [54]:
pygal_data  = [(histo_data.loc[x, 0], histo_data.loc[x, "sinad_5w_4ohm_integer"], histo_data.loc[x, "sinad_5w_4ohm_integer"] + 1) for x in list(histo_data.index)]

In [55]:
chart = pygal.Histogram()
chart.config.title = f"sinad_5w_4ohm_integer histogram (all values rounded down to nearest integer). Mean: {round(data.sinad_5w_4ohm.mean(), 2)}"
chart.config.legend_at_bottom = True
chart.add('sinad_5w_4ohm',pygal_data)

# Export to svg
chart.render_to_file("./chart-exports/sinad-5watt-4ohm-histogram.svg")

<img src="chart-exports/sinad-5watt-4ohm-histogram.svg" width="1000"/>

### Price usd vs sinad scatter plot

In [56]:
max_price_filter = 7500
first_quartile = data[(data.thd_quartile == "quartile 1") & (data.price_usd < max_price_filter)]
second_quartile = data[(data.thd_quartile == "quartile 2") & (data.price_usd < max_price_filter)]
third_quartile = data[(data.thd_quartile == "quartile 3") & (data.price_usd < max_price_filter)]
fourth_quartile = data[(data.thd_quartile == "quartile 4") & (data.price_usd < max_price_filter)]

chart = pygal.XY(stroke=False)

chart.title = f"price_usd (y axis) vs sinad_5w_4ohm (x axis) where price_usd < ${max_price_filter} usd,  grouped by thd @ 1% category"
chart.config.legend_at_bottom = True

# Add data
chart.add(quartile_dict["Q1"], list(zip(first_quartile.sinad_5w_4ohm, first_quartile.price_usd)))
chart.add(quartile_dict["Q2"], list(zip(second_quartile.sinad_5w_4ohm, second_quartile.price_usd)))
chart.add(quartile_dict["Q3"], list(zip(third_quartile.sinad_5w_4ohm, third_quartile.price_usd)))
chart.add(quartile_dict["Q4"], list(zip(fourth_quartile.sinad_5w_4ohm, fourth_quartile.price_usd)))

# Export to svg
chart.render_to_file("./chart-exports/price-usd-vs-sinad-5watt-4ohm-scatterplot.svg")

<img src="chart-exports/price-usd-vs-sinad-5watt-4ohm-scatterplot.svg" width="1000"/>